In [1]:
import pandas as pd

In [2]:
base_path = '../../data/01_additional_data/'
matches_file_name = 'MATCHES_2013-2019.xlsx'
festivities_file_name = 'festivities_calendar_Madrid_2013-2020.csv'

## Matches

In [3]:
# Read matches
matches = pd.read_excel(base_path+matches_file_name,parse_dates = ['Date'] )
matches.head()
matches.columns =  [col.lower() for col in matches.columns]

In [4]:
df_matches = matches.groupby('date')['matches_madrid'].sum().reset_index()

In [5]:
# Filter
df_matches = matches[matches['matches_madrid']>0][['date','matches_madrid']]

In [6]:
df_matches[df_matches['date']<= '2019-09-30'].shape

(1086, 2)

## Festivities

In [7]:
df_public_holidays = pd.read_csv(base_path+festivities_file_name, encoding = "ISO-8859-1", sep = ';', parse_dates = ['Dia'])

In [8]:
df_public_holidays = df_public_holidays.loc[df_public_holidays['laborable / festivo / domingo festivo']=='festivo','Dia'].to_frame()

In [9]:
df_public_holidays['festivo'] = 1

In [10]:
df_public_holidays.head()

,Dia,festivo
0,2013-01-01,1
6,2013-07-01,1
76,2013-03-18,1
86,2013-03-28,1
87,2013-03-29,1


In [11]:
df_public_holidays[df_public_holidays['Dia']<= '2019-09-30'].shape

(102, 2)

## Adding it to the transactions dataframe

In [12]:
# Reading the file
input_path = "../../data/03_processed/" + "filtered_transactions_clean"
filtered_file_name="c1-filtered_transactions.csv"
sep=";"
df = pd.read_csv(input_path, sep=sep, parse_dates = ['order_date'])

In [13]:
# Grouping by product and date:
df = df.groupby(['order_date','product'])['units_ordered'].sum().reset_index()
df.shape

(42910, 3)

### Adding Public holidays

In [14]:
df_holidays = pd.merge(df, df_public_holidays, how = 'left', left_on='order_date', right_on='Dia')

In [15]:
df_holidays[df_holidays['festivo']==1]['order_date'].unique().shape

(102,)

In [16]:
df_holidays.head()

,order_date,product,units_ordered,Dia,festivo
0,2008-01-01,baguette,86.0,NaT,NaN
1,2008-01-01,croissant petit,178.0,NaT,NaN
2,2008-01-01,croissant simple,137.0,NaT,NaN
3,2008-01-01,milhojas frambuesa,12.0,NaT,NaN
4,2008-01-01,mousse tres chocolates,2.0,NaT,NaN


In [17]:
df_holidays.shape

(42910, 5)

### Adding matches

In [18]:
df_holidays_matches = pd.merge(df_holidays, df_matches, how = 'left', left_on='order_date', right_on='date')

In [19]:
df_holidays_matches.head()

,order_date,product,units_ordered,Dia,festivo,date,matches_madrid
0,2008-01-01,baguette,86.0,NaT,NaN,NaT,NaN
1,2008-01-01,croissant petit,178.0,NaT,NaN,NaT,NaN
2,2008-01-01,croissant simple,137.0,NaT,NaN,NaT,NaN
3,2008-01-01,milhojas frambuesa,12.0,NaT,NaN,NaT,NaN
4,2008-01-01,mousse tres chocolates,2.0,NaT,NaN,NaT,NaN


In [20]:
df_holidays_matches[df_holidays_matches['matches_madrid']==1]['order_date'].unique().shape

(612,)

In [21]:
df_holidays_matches = df_holidays_matches[['order_date', 'product', 'units_ordered', 'festivo','matches_madrid']]

In [22]:
df_holidays_matches.fillna(0, inplace = True)

In [23]:
df_holidays_matches

,order_date,product,units_ordered,festivo,matches_madrid
0,2008-01-01,baguette,86.0,0.0,0.0
1,2008-01-01,croissant petit,178.0,0.0,0.0
2,2008-01-01,croissant simple,137.0,0.0,0.0
3,2008-01-01,milhojas frambuesa,12.0,0.0,0.0
4,2008-01-01,mousse tres chocolates,2.0,0.0,0.0
5,2008-01-01,palmera chocolate,46.0,0.0,0.0
6,2008-01-01,postre de fresas y mascarpone,0.0,0.0,0.0
7,2008-01-01,tarta de manzana,0.0,0.0,0.0
8,2008-01-01,tarta opera,4.0,0.0,0.0
9,2008-01-01,tortel,256.0,0.0,0.0


In [24]:
exit_path = "../../data/03_processed/" + "time_series.csv"
df_holidays_matches.to_csv(exit_path)